In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing import sequence
import pickle

Using TensorFlow backend.


In [2]:
cd /content/drive/My Drive/Case_study_2

/content/drive/My Drive/Case_study_2


In [0]:
train_df = pd.read_csv('data.tsv',delimiter='\t',names = ['Q_id','Question','Answer','Target','Respose_number'])

In [4]:
cd /content/drive/My Drive/Case_study_2/For_running_final

/content/drive/My Drive/Case_study_2/For_running_final


In [0]:
#loaing Data
#sample csv data have 3000 data points from orginal data set
#train_df = pd.read_csv('sample_data.csv')

In [5]:
train_df.head()

,Q_id,Question,Answer,Target,Respose_number
0,0,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,0,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,0,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,0,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3
4,0,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4


## Function to make predictions made on Raw Data

In [0]:
def final_fun_1(X):
    data = X.copy()
    preprocessed_Questions = []
    for sentance in (data['Question'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Questions.append(sent)
    
    preprocessed_Answers = []
    for sentance in (data['Answer'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Answers.append(sent)
        
    data['Question'] = preprocessed_Questions
    data['Answer'] = preprocessed_Answers
    
    
    Question_keywords = np.load(r'Questin_Answer_10000_keywords/Question_keywords.npy',allow_pickle = True)
    Answer_keywords = np.load(r'Questin_Answer_10000_keywords/Answer_keywords.npy',allow_pickle = True)
    
    que_which_has_ly_keywords = []
    for sentance in data['Question']:
        sent = ' '.join(e for e in sentance.split() if e in Question_keywords)
        que_which_has_ly_keywords.append(sent)
    
     
    ans_which_has_ly_keywords = []
    for sentance in data['Answer']:
        sent = ' '.join(e for e in sentance.split() if e in Answer_keywords)
        ans_which_has_ly_keywords.append(sent)
        
    data['Question'] = que_which_has_ly_keywords
    data['Answer'] = ans_which_has_ly_keywords
        
        
    #Setting numerical value to each word in questions
    ecoded_ques = []
    for train_ques in data['Question']:
        ecoded_ques.append(list(map(lambda x: list(Question_keywords).index(x)+1,train_ques.split(" "))))
    
    ecoded_ans = []
    for train_ans in data['Answer']:
        ecoded_ans.append(list(map(lambda x: list(Answer_keywords).index(x)+1,train_ans.split(" "))))
    
    data['Question_vector'] = ecoded_ques
    data['Answer_vector'] = ecoded_ans
    
    
    #truncate and/or pad input sequences
    Max_Question_length = 24
    ques_vect_1 = sequence.pad_sequences(list(data.Question_vector), maxlen=Max_Question_length)
    ques_vect = [np.hstack((x,10001)) for x in ques_vect_1]
        
    # truncate and/or pad input sequences
    Max_Answer_length = 163
    ans_vect = sequence.pad_sequences(data.Answer_vector, maxlen=Max_Answer_length)
    
    
    #load config & weight of best model
    with open('Final_model_weight/Best_model/config_9.p', 'rb') as fp:
        config = pickle.load(fp)

    with open('Final_model_weight/Best_model//Best_weights_9.p', 'rb') as fp:
        weights = pickle.load(fp)

    #loading weights to model
    import tensorflow as tf
    model = tf.keras.Model.from_config(config)
    model.set_weights(weights)
    
    ques_vect_ds = list(tf.data.Dataset.from_tensor_slices((ques_vect)).batch(1024).as_numpy_iterator())
    ans_vect_ds = list(tf.data.Dataset.from_tensor_slices((ans_vect)).batch(1024).as_numpy_iterator())
    

    @tf.function
    def predict(input_vec):
        '''Taks in a batch of vector ,pass it into model and returns the predicted value'''
        predictions = model(input_vec, training=False)
        return predictions

    #stack the output of each batch
    def predicted_y(ques_vect,ans_vect):
        '''Takes in the a two vector(Question,answe vecror) and returns the predicted result'''
        ls = []
        for batch in range(len(ques_vect)):
            test_ques_vec = ques_vect[batch]
            test_ans_vec = ans_vect[batch]
            ls.append(predict([test_ques_vec,test_ans_vec]))

        pred_y = ls[0]
        for i in range(1,len(ls)):
            pred_y = np.vstack((pred_y,ls[i]))
        return pred_y
    
    predicted_y = predicted_y(ques_vect_ds,ans_vect_ds)
    
    for_output = np.zeros_like(predicted_y,dtype = np.int32)
    for i in range(0,predicted_y.shape[0],10):
        maxi = np.argmax(predicted_y[i:i+10])
        for_output[i+maxi] = 1
        

    return np.array(for_output[:,0]),np.array(predicted_y[:,0])

In [8]:
binary_output,probablity_output = final_fun_1(train_df.iloc[:2000,:])
print("Binary Value output : \n", binary_output)
print("Probablity Value output : \n", probablity_output)

Binary Value output : 
 [0 0 0 ... 0 0 0]
Probablity Value output : 
 [0.13544637 0.00307176 0.39006913 ... 0.00245118 0.01382267 0.00612232]


## Function  which returns final_metric computed on X ( Raw Data) and Y (target variable)

In [0]:
def final_fun_2(X):
    data = X.copy()
    preprocessed_Questions = []
    for sentance in (data['Question'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Questions.append(sent)
    
    preprocessed_Answers = []
    for sentance in (data['Answer'].values):
        sent = re.sub('[^A-Za-z0-9]+', ' ', sentance)
        sent = ' '.join(e.lower() for e in sent.split())
        preprocessed_Answers.append(sent)
        
    data['Question'] = preprocessed_Questions
    data['Answer'] = preprocessed_Answers
    
    
    Question_keywords = np.load(r'Questin_Answer_10000_keywords/Question_keywords.npy',allow_pickle = True)
    Answer_keywords = np.load(r'Questin_Answer_10000_keywords/Answer_keywords.npy',allow_pickle = True)
    
    que_which_has_ly_keywords = []
    for sentance in data['Question']:
        sent = ' '.join(e for e in sentance.split() if e in Question_keywords)
        que_which_has_ly_keywords.append(sent)
    
     
    ans_which_has_ly_keywords = []
    for sentance in data['Answer']:
        sent = ' '.join(e for e in sentance.split() if e in Answer_keywords)
        ans_which_has_ly_keywords.append(sent)
        
    data['Question'] = que_which_has_ly_keywords
    data['Answer'] = ans_which_has_ly_keywords
        
    
    #Setting numerical value to each word in questions
    ecoded_ques = []
    for train_ques in data['Question']:
        ecoded_ques.append(list(map(lambda x: list(Question_keywords).index(x)+1,train_ques.split(" "))))
    
    ecoded_ans = []
    for train_ans in data['Answer']:
        ecoded_ans.append(list(map(lambda x: list(Answer_keywords).index(x)+1,train_ans.split(" "))))
    
    data['Question_vector'] = ecoded_ques
    data['Answer_vector'] = ecoded_ans
    
    #truncate and/or pad input sequences
    Max_Question_length = 24
    ques_vect_1 = sequence.pad_sequences(list(data.Question_vector), maxlen=Max_Question_length)
    ques_vect = [np.hstack((x,10001)) for x in ques_vect_1]
    
    
    # truncate and/or pad input sequences
    Max_Answer_length = 163
    ans_vect = sequence.pad_sequences(data.Answer_vector, maxlen=Max_Answer_length)
    
    
    #load config & weight of best model
    with open('Final_model_weight/Best_model/config_9.p', 'rb') as fp:
        config = pickle.load(fp)

    with open('Final_model_weight/Best_model//Best_weights_9.p', 'rb') as fp:
        weights = pickle.load(fp)

    
    #loading weights to model
    import tensorflow as tf
    model = tf.keras.Model.from_config(config)
    model.set_weights(weights)
    
    
    ques_vect_ds = list(tf.data.Dataset.from_tensor_slices((ques_vect)).batch(1024).as_numpy_iterator())
    ans_vect_ds = list(tf.data.Dataset.from_tensor_slices((ans_vect)).batch(1024).as_numpy_iterator())
    target_ds = list(tf.data.Dataset.from_tensor_slices((data.Target)).batch(1024).as_numpy_iterator())

    loss = tf.keras.metrics.BinaryCrossentropy(name='loss')
    loss.reset_states()
    predicted_y = []
    
    def test_step(input_vec, labels):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(input_vec, training=False)
        predicted_y.append(predictions.numpy()[:,0])
        loss(labels, predictions)

    def calculate_loss(ques_vect_ds,ans_vect_ds,target_ds):
        for batch in range(len(ques_vect_ds)):
            test_ques_vec = ques_vect_ds[batch]
            test_ans_vec = ans_vect_ds[batch]
            target = target_ds[batch]
            test_step([test_ques_vec,test_ans_vec], target)

    calculate_loss(ques_vect_ds,ans_vect_ds,target_ds)

    #calculate MRR
    rr=0
    for i in range(0,predicted_y[0].shape[0],10):
        rr +=1/(list(np.argsort(np.array(predicted_y[0]).reshape(-1,1)[i:i+10][:,0])[::-1]).index(np.argmax(target_ds[0][i:i+10]))+1)
    MRR = rr/(predicted_y[0].shape[0]/10)


    return loss.result().numpy(),MRR

In [10]:
loss,MRR= final_fun_2(train_df.iloc[:2000,:])
print("Loss :" , loss)
print("MRR : ",MRR)

Loss : 0.2877097
MRR :  0.4420107886904762
